<a href="https://colab.research.google.com/github/Seyedyt/ML_Projectphase1/blob/main/project_phase1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Preparation
We'll start by importing the necessary libraries and preparing the CIFAR-10 dataset.

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score
from torch.utils.data import DataLoader, Subset
import numpy as np
import math

In [9]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Transformations for the CIFAR-10 dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [10]:
# Download CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Create a very small subset of the CIFAR-10 dataset for quick testing
subset_indices = np.random.choice(len(train_dataset), 5000, replace=False)
small_train_dataset = Subset(train_dataset, subset_indices)

print(f'Original training dataset size: {len(train_dataset)}')
print(f'Smaller training dataset size: {len(small_train_dataset)}')

Files already downloaded and verified
Files already downloaded and verified
Original training dataset size: 50000
Smaller training dataset size: 5000


In [11]:
# Function to split dataset into shards and slices
def split_dataset(dataset, num_shards, num_slices):
    shard_size = len(dataset) // num_shards
    shards = [Subset(dataset, range(i * shard_size, (i + 1) * shard_size)) for i in range(num_shards)]
    slices = []
    for shard in shards:
        slice_size = len(shard) // num_slices
        shard_slices = [Subset(shard, range(j * slice_size, (j + 1) * slice_size)) for j in range(num_slices)]
        slices.append(shard_slices)
    return slices

## Model Training
Next, we'll define a function to train the model using the slices from each shard.

In [12]:
def train_model_on_slices(slices, epochs_per_slice):
    trained_models = []
    criterion = nn.CrossEntropyLoss()
    print('Training...')

    print(f'Number of shards (slices): {len(slices)}')
    counter1 = 0  # Counter for shards
    counter2 = 0  # Counter for slices

    for shard in slices:
        counter1 += 1
        print(f'Training shard {counter1}/{len(slices)}')

        model = models.resnet18(pretrained=True)
        # Modify ResNet-18 to add fully connected layers
        num_features = model.fc.in_features
        model.fc = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 10)  # CIFAR-10 has 10 classes
        )
        model = model.to(device)
        optimizer = optim.Adam(model.parameters(), lr=0.001)

        print(f'Number of slices in shard {counter1}: {len(shard)}')
        for slice in shard:
            counter2 += 1
            print(f'Training slice {counter2} in shard {counter1}')

            dataloader = DataLoader(slice, batch_size=64, shuffle=True)
            model.train()
            for epoch in range(epochs_per_slice):
                for images, labels in dataloader:
                    images, labels = images.to(device), labels.to(device)
                    optimizer.zero_grad()
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()

        trained_models.append(model)

    return trained_models


## Model Aggregation
We'll aggregate the trained models by averaging their predictions.

In [13]:
def aggregate_models(trained_models, dataloader):
    all_probabilities = []
    all_predictions = []
    all_labels = []
    loop_count = 0  # Counter for the number of iterations
    dataloader_size = len(dataloader)
    print(f'DataLoader size (number of batches): {dataloader_size}')

    with torch.no_grad():
        for images, labels in dataloader:
            print(f'Iteration: {loop_count}')
            loop_count += 1  # Increment counter
            images = images.to(device)
            outputs = [model(images).cpu() for model in trained_models]
            avg_output = sum(outputs) / len(outputs)
            probabilities = nn.Softmax(dim=1)(avg_output)  # Apply softmax to get probabilities
            _, predictions = torch.max(probabilities, 1)
            all_probabilities.extend(probabilities.numpy())
            all_predictions.extend(predictions.numpy())
            all_labels.extend(labels.numpy())

    print(f'The for loop in aggregate_models ran {loop_count} times.')
    return all_probabilities, all_predictions, all_labels


## Evaluation

In [14]:
def evaluate_model(probabilities, predictions, labels):
    f1 = f1_score(labels, predictions, average='macro')
    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, average='macro')
    recall = recall_score(labels, predictions, average='macro')
    # For AUROC, we need probabilities
    auroc = roc_auc_score(labels, probabilities, multi_class='ovr')
    return f1, accuracy, precision, recall, auroc

# Training and evaluation process for different configurations
S_values = [20]
R_values = [20]
epochs_per_slice = 1  # Number of epochs to train per slice for quick testing

# Store the results
results = []

for S in S_values:
    for R in R_values:
        print(f'Training with S={S}, R={R}...')
        slices = split_dataset(train_dataset, S, R)
        print('slice completed')
        trained_models = train_model_on_slices(slices, epochs_per_slice)
        print('train completed')
        test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
        print('loading completed')

        # Aggregate models and evaluate
        probabilities, predictions, labels = aggregate_models(trained_models, test_loader)
        print('aggregating completed')
        metrics = evaluate_model(probabilities, predictions, labels)

        # Store results
        results.append({
            'S': S,
            'R': R,
            'F1 Score': metrics[0],
            'Accuracy': metrics[1],
            'Precision': metrics[2],
            'Recall': metrics[3],
            'AUROC': metrics[4]
        })

# Display results
for result in results:
    print(f'Configuration (S={result["S"]}, R={result["R"]}):')
    print(f'F1 Score: {result["F1 Score"]}')
    print(f'Accuracy: {result["Accuracy"]}')
    print(f'Precision: {result["Precision"]}')
    print(f'Recall: {result["Recall"]}')
    print(f'AUROC: {result["AUROC"]}')
    print('----------------------------')

Training with S=20, R=20...
slice completed
Training...
Number of shards (slices): 20
Training shard 1/20


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Number of slices in shard 1: 20
Training slice 1 in shard 1
Training slice 2 in shard 1
Training slice 3 in shard 1
Training slice 4 in shard 1
Training slice 5 in shard 1
Training slice 6 in shard 1
Training slice 7 in shard 1
Training slice 8 in shard 1
Training slice 9 in shard 1
Training slice 10 in shard 1
Training slice 11 in shard 1
Training slice 12 in shard 1
Training slice 13 in shard 1
Training slice 14 in shard 1
Training slice 15 in shard 1
Training slice 16 in shard 1
Training slice 17 in shard 1
Training slice 18 in shard 1
Training slice 19 in shard 1
Training slice 20 in shard 1
Training shard 2/20
Number of slices in shard 2: 20
Training slice 21 in shard 2
Training slice 22 in shard 2
Training slice 23 in shard 2
Training slice 24 in shard 2
Training slice 25 in shard 2
Training slice 26 in shard 2
Training slice 27 in shard 2
Training slice 28 in shard 2
Training slice 29 in shard 2
Training slice 30 in shard 2
Training slice 31 in shard 2
Training slice 32 in shard

In [15]:
# Training and evaluation process for different configurations
S_values = [10]
R_values = [20]
epochs_per_slice = 1  # Number of epochs to train per slice for quick testing

for S in S_values:
    for R in R_values:
        print(f'Training with S={S}, R={R}...')
        slices = split_dataset(small_train_dataset, S, R)
        print('slice completed')
        trained_models = train_model_on_slices(slices, epochs_per_slice)
        print('train completed')
        test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
        print('loading completed')

        # Aggregate models and evaluate
        probabilities, predictions, labels = aggregate_models(trained_models, test_loader)
        print('aggregating completed')
        metrics = evaluate_model(probabilities, predictions, labels)

        # Store results
        results.append({
            'S': S,
            'R': R,
            'F1 Score': metrics[0],
            'Accuracy': metrics[1],
            'Precision': metrics[2],
            'Recall': metrics[3],
            'AUROC': metrics[4]
        })

Training with S=10, R=20...
slice completed
Training...
Number of shards (slices): 10
Training shard 1/10


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Number of slices in shard 1: 20
Training slice 1 in shard 1
Training slice 2 in shard 1
Training slice 3 in shard 1
Training slice 4 in shard 1
Training slice 5 in shard 1
Training slice 6 in shard 1
Training slice 7 in shard 1
Training slice 8 in shard 1
Training slice 9 in shard 1
Training slice 10 in shard 1
Training slice 11 in shard 1
Training slice 12 in shard 1
Training slice 13 in shard 1
Training slice 14 in shard 1
Training slice 15 in shard 1
Training slice 16 in shard 1
Training slice 17 in shard 1
Training slice 18 in shard 1
Training slice 19 in shard 1
Training slice 20 in shard 1
Training shard 2/10
Number of slices in shard 2: 20
Training slice 21 in shard 2
Training slice 22 in shard 2
Training slice 23 in shard 2
Training slice 24 in shard 2
Training slice 25 in shard 2
Training slice 26 in shard 2
Training slice 27 in shard 2
Training slice 28 in shard 2
Training slice 29 in shard 2
Training slice 30 in shard 2
Training slice 31 in shard 2
Training slice 32 in shard

In [17]:
# Training and evaluation process for different configurations
S_values = [5]
R_values = [20]
epochs_per_slice = 1  # Number of epochs to train per slice for quick testing

for S in S_values:
    for R in R_values:
        print(f'Training with S={S}, R={R}...')
        slices = split_dataset(small_train_dataset, S, R)
        print('slice completed')
        trained_models = train_model_on_slices(slices, epochs_per_slice)
        print('train completed')
        test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
        print('loading completed')

        # Aggregate models and evaluate
        probabilities, predictions, labels = aggregate_models(trained_models, test_loader)
        print('aggregating completed')
        metrics = evaluate_model(probabilities, predictions, labels)

        # Store results
        results.append({
            'S': S,
            'R': R,
            'F1 Score': metrics[0],
            'Accuracy': metrics[1],
            'Precision': metrics[2],
            'Recall': metrics[3],
            'AUROC': metrics[4]
        })

Training with S=5, R=20...
slice completed
Training...
Number of shards (slices): 5
Training shard 1/5


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Number of slices in shard 1: 20
Training slice 1 in shard 1
Training slice 2 in shard 1
Training slice 3 in shard 1
Training slice 4 in shard 1
Training slice 5 in shard 1
Training slice 6 in shard 1
Training slice 7 in shard 1
Training slice 8 in shard 1
Training slice 9 in shard 1
Training slice 10 in shard 1
Training slice 11 in shard 1
Training slice 12 in shard 1
Training slice 13 in shard 1
Training slice 14 in shard 1
Training slice 15 in shard 1
Training slice 16 in shard 1
Training slice 17 in shard 1
Training slice 18 in shard 1
Training slice 19 in shard 1
Training slice 20 in shard 1
Training shard 2/5
Number of slices in shard 2: 20
Training slice 21 in shard 2
Training slice 22 in shard 2
Training slice 23 in shard 2
Training slice 24 in shard 2
Training slice 25 in shard 2
Training slice 26 in shard 2
Training slice 27 in shard 2
Training slice 28 in shard 2
Training slice 29 in shard 2
Training slice 30 in shard 2
Training slice 31 in shard 2
Training slice 32 in shard 

In [19]:
# Training and evaluation process for different configurations
S_values = [20]
R_values = [10]
epochs_per_slice = 1  # Number of epochs to train per slice for quick testing

for S in S_values:
    for R in R_values:
        print(f'Training with S={S}, R={R}...')
        slices = split_dataset(small_train_dataset, S, R)
        print('slice completed')
        trained_models = train_model_on_slices(slices, epochs_per_slice)
        print('train completed')
        test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
        print('loading completed')

        # Aggregate models and evaluate
        probabilities, predictions, labels = aggregate_models(trained_models, test_loader)
        print('aggregating completed')
        metrics = evaluate_model(probabilities, predictions, labels)

        # Store results
        results.append({
            'S': S,
            'R': R,
            'F1 Score': metrics[0],
            'Accuracy': metrics[1],
            'Precision': metrics[2],
            'Recall': metrics[3],
            'AUROC': metrics[4]
        })

Training with S=20, R=10...
slice completed
Training...
Number of shards (slices): 20
Training shard 1/20


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Number of slices in shard 1: 10
Training slice 1 in shard 1
Training slice 2 in shard 1
Training slice 3 in shard 1
Training slice 4 in shard 1
Training slice 5 in shard 1
Training slice 6 in shard 1
Training slice 7 in shard 1
Training slice 8 in shard 1
Training slice 9 in shard 1
Training slice 10 in shard 1
Training shard 2/20
Number of slices in shard 2: 10
Training slice 11 in shard 2
Training slice 12 in shard 2
Training slice 13 in shard 2
Training slice 14 in shard 2
Training slice 15 in shard 2
Training slice 16 in shard 2
Training slice 17 in shard 2
Training slice 18 in shard 2
Training slice 19 in shard 2
Training slice 20 in shard 2
Training shard 3/20
Number of slices in shard 3: 10
Training slice 21 in shard 3
Training slice 22 in shard 3
Training slice 23 in shard 3
Training slice 24 in shard 3
Training slice 25 in shard 3
Training slice 26 in shard 3
Training slice 27 in shard 3
Training slice 28 in shard 3
Training slice 29 in shard 3
Training slice 30 in shard 3
Tra

In [21]:
# Training and evaluation process for different configurations
S_values = [10]
R_values = [10]
epochs_per_slice = 1  # Number of epochs to train per slice for quick testing

for S in S_values:
    for R in R_values:
        print(f'Training with S={S}, R={R}...')
        slices = split_dataset(small_train_dataset, S, R)
        print('slice completed')
        trained_models = train_model_on_slices(slices, epochs_per_slice)
        print('train completed')
        test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
        print('loading completed')

        # Aggregate models and evaluate
        probabilities, predictions, labels = aggregate_models(trained_models, test_loader)
        print('aggregating completed')
        metrics = evaluate_model(probabilities, predictions, labels)

        # Store results
        results.append({
            'S': S,
            'R': R,
            'F1 Score': metrics[0],
            'Accuracy': metrics[1],
            'Precision': metrics[2],
            'Recall': metrics[3],
            'AUROC': metrics[4]
        })

Training with S=10, R=10...
slice completed
Training...
Number of shards (slices): 10
Training shard 1/10


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Number of slices in shard 1: 10
Training slice 1 in shard 1
Training slice 2 in shard 1
Training slice 3 in shard 1
Training slice 4 in shard 1
Training slice 5 in shard 1
Training slice 6 in shard 1
Training slice 7 in shard 1
Training slice 8 in shard 1
Training slice 9 in shard 1
Training slice 10 in shard 1
Training shard 2/10
Number of slices in shard 2: 10
Training slice 11 in shard 2
Training slice 12 in shard 2
Training slice 13 in shard 2
Training slice 14 in shard 2
Training slice 15 in shard 2
Training slice 16 in shard 2
Training slice 17 in shard 2
Training slice 18 in shard 2
Training slice 19 in shard 2
Training slice 20 in shard 2
Training shard 3/10
Number of slices in shard 3: 10
Training slice 21 in shard 3
Training slice 22 in shard 3
Training slice 23 in shard 3
Training slice 24 in shard 3
Training slice 25 in shard 3
Training slice 26 in shard 3
Training slice 27 in shard 3
Training slice 28 in shard 3
Training slice 29 in shard 3
Training slice 30 in shard 3
Tra

In [23]:
# Training and evaluation process for different configurations
S_values = [5]
R_values = [10]
epochs_per_slice = 1  # Number of epochs to train per slice for quick testing

for S in S_values:
    for R in R_values:
        print(f'Training with S={S}, R={R}...')
        slices = split_dataset(small_train_dataset, S, R)
        print('slice completed')
        trained_models = train_model_on_slices(slices, epochs_per_slice)
        print('train completed')
        test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
        print('loading completed')

        # Aggregate models and evaluate
        probabilities, predictions, labels = aggregate_models(trained_models, test_loader)
        print('aggregating completed')
        metrics = evaluate_model(probabilities, predictions, labels)

        # Store results
        results.append({
            'S': S,
            'R': R,
            'F1 Score': metrics[0],
            'Accuracy': metrics[1],
            'Precision': metrics[2],
            'Recall': metrics[3],
            'AUROC': metrics[4]
        })

Training with S=5, R=10...
slice completed
Training...
Number of shards (slices): 5
Training shard 1/5


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Number of slices in shard 1: 10
Training slice 1 in shard 1
Training slice 2 in shard 1
Training slice 3 in shard 1
Training slice 4 in shard 1
Training slice 5 in shard 1
Training slice 6 in shard 1
Training slice 7 in shard 1
Training slice 8 in shard 1
Training slice 9 in shard 1
Training slice 10 in shard 1
Training shard 2/5
Number of slices in shard 2: 10
Training slice 11 in shard 2
Training slice 12 in shard 2
Training slice 13 in shard 2
Training slice 14 in shard 2
Training slice 15 in shard 2
Training slice 16 in shard 2
Training slice 17 in shard 2
Training slice 18 in shard 2
Training slice 19 in shard 2
Training slice 20 in shard 2
Training shard 3/5
Number of slices in shard 3: 10
Training slice 21 in shard 3
Training slice 22 in shard 3
Training slice 23 in shard 3
Training slice 24 in shard 3
Training slice 25 in shard 3
Training slice 26 in shard 3
Training slice 27 in shard 3
Training slice 28 in shard 3
Training slice 29 in shard 3
Training slice 30 in shard 3
Train

In [25]:
# Training and evaluation process for different configurations
S_values = [20]
R_values = [5]
epochs_per_slice = 1  # Number of epochs to train per slice for quick testing

for S in S_values:
    for R in R_values:
        print(f'Training with S={S}, R={R}...')
        slices = split_dataset(small_train_dataset, S, R)
        print('slice completed')
        trained_models = train_model_on_slices(slices, epochs_per_slice)
        print('train completed')
        test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
        print('loading completed')

        # Aggregate models and evaluate
        probabilities, predictions, labels = aggregate_models(trained_models, test_loader)
        print('aggregating completed')
        metrics = evaluate_model(probabilities, predictions, labels)

        # Store results
        results.append({
            'S': S,
            'R': R,
            'F1 Score': metrics[0],
            'Accuracy': metrics[1],
            'Precision': metrics[2],
            'Recall': metrics[3],
            'AUROC': metrics[4]
        })

Training with S=20, R=5...
slice completed
Training...
Number of shards (slices): 20
Training shard 1/20


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Number of slices in shard 1: 5
Training slice 1 in shard 1
Training slice 2 in shard 1
Training slice 3 in shard 1
Training slice 4 in shard 1
Training slice 5 in shard 1
Training shard 2/20
Number of slices in shard 2: 5
Training slice 6 in shard 2
Training slice 7 in shard 2
Training slice 8 in shard 2
Training slice 9 in shard 2
Training slice 10 in shard 2
Training shard 3/20
Number of slices in shard 3: 5
Training slice 11 in shard 3
Training slice 12 in shard 3
Training slice 13 in shard 3
Training slice 14 in shard 3
Training slice 15 in shard 3
Training shard 4/20
Number of slices in shard 4: 5
Training slice 16 in shard 4
Training slice 17 in shard 4
Training slice 18 in shard 4
Training slice 19 in shard 4
Training slice 20 in shard 4
Training shard 5/20
Number of slices in shard 5: 5
Training slice 21 in shard 5
Training slice 22 in shard 5
Training slice 23 in shard 5
Training slice 24 in shard 5
Training slice 25 in shard 5
Training shard 6/20
Number of slices in shard 6: 

In [27]:
# Training and evaluation process for different configurations
S_values = [10]
R_values = [5]
epochs_per_slice = 1  # Number of epochs to train per slice for quick testing

for S in S_values:
    for R in R_values:
        print(f'Training with S={S}, R={R}...')
        slices = split_dataset(small_train_dataset, S, R)
        print('slice completed')
        trained_models = train_model_on_slices(slices, epochs_per_slice)
        print('train completed')
        test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
        print('loading completed')

        # Aggregate models and evaluate
        probabilities, predictions, labels = aggregate_models(trained_models, test_loader)
        print('aggregating completed')
        metrics = evaluate_model(probabilities, predictions, labels)

        # Store results
        results.append({
            'S': S,
            'R': R,
            'F1 Score': metrics[0],
            'Accuracy': metrics[1],
            'Precision': metrics[2],
            'Recall': metrics[3],
            'AUROC': metrics[4]
        })

Training with S=10, R=5...
slice completed
Training...
Number of shards (slices): 10
Training shard 1/10


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Number of slices in shard 1: 5
Training slice 1 in shard 1
Training slice 2 in shard 1
Training slice 3 in shard 1
Training slice 4 in shard 1
Training slice 5 in shard 1
Training shard 2/10
Number of slices in shard 2: 5
Training slice 6 in shard 2
Training slice 7 in shard 2
Training slice 8 in shard 2
Training slice 9 in shard 2
Training slice 10 in shard 2
Training shard 3/10
Number of slices in shard 3: 5
Training slice 11 in shard 3
Training slice 12 in shard 3
Training slice 13 in shard 3
Training slice 14 in shard 3
Training slice 15 in shard 3
Training shard 4/10
Number of slices in shard 4: 5
Training slice 16 in shard 4
Training slice 17 in shard 4
Training slice 18 in shard 4
Training slice 19 in shard 4
Training slice 20 in shard 4
Training shard 5/10
Number of slices in shard 5: 5
Training slice 21 in shard 5
Training slice 22 in shard 5
Training slice 23 in shard 5
Training slice 24 in shard 5
Training slice 25 in shard 5
Training shard 6/10
Number of slices in shard 6: 

In [28]:
# Display results
for result in results:
    print(f'Configuration (S={result["S"]}, R={result["R"]}):')
    print(f'F1 Score: {result["F1 Score"]}')
    print(f'Accuracy: {result["Accuracy"]}')
    print(f'Precision: {result["Precision"]}')
    print(f'Recall: {result["Recall"]}')
    print(f'AUROC: {result["AUROC"]}')
    print('----------------------------')

Configuration (S=20, R=20):
F1 Score: 0.7832434319244326
Accuracy: 0.7847
Precision: 0.7833014832665263
Recall: 0.7846999999999998
AUROC: 0.9736947777777777
----------------------------
Configuration (S=10, R=20):
F1 Score: 0.57382625001074
Accuracy: 0.5989
Precision: 0.6115152286026596
Recall: 0.5989000000000001
AUROC: 0.9287558722222222
----------------------------
Configuration (S=5, R=20):
F1 Score: 0.6775472950272172
Accuracy: 0.6835
Precision: 0.6933201279738412
Recall: 0.6835
AUROC: 0.9513980666666667
----------------------------
Configuration (S=20, R=10):
F1 Score: 0.5739196599644721
Accuracy: 0.6014
Precision: 0.6339897491614332
Recall: 0.6013999999999999
AUROC: 0.9280301944444445
----------------------------
Configuration (S=10, R=10):
F1 Score: 0.6371376293713931
Accuracy: 0.651
Precision: 0.675171207749908
Recall: 0.651
AUROC: 0.9460089444444444
----------------------------
Configuration (S=5, R=10):
F1 Score: 0.6674826718293632
Accuracy: 0.6687
Precision: 0.67233830069024